In [42]:
import numpy as np
from numpy import linalg as LA
from numpy.linalg import inv
from scipy.linalg import rq
import math

In [43]:
# X = np.array([
#     [4.74, 4.74, 0],
#     [7.11, 7.11, 0],
#     [7.11, 11.85, 0],
#     [11.85, 11.85, 4.74],
#     [11.85, 7.11, 4.74],
#     [11.85, 4.74, 7.11]
# ]
# )
X = np.array([
    [2,2,0],
    [3,5,0],
    [5,5,2],
    [5,2,3],
    [5,6,3],
    [5,4,1]
]
)


In [44]:
# x = np.array([
#     [356, 836],
#     [450, 815],
#     [581, 886],
#     [682, 700],
#     [556, 804],
#     [528, 532]
# ]
# )
x = np.array([
    [356, 836],
    [581, 886],
    [682, 700],
    [528, 532],
    [789, 694],
    [580, 702]
]
)

In [45]:
def normalizing_params(pts):
    centroid=np.mean(pts,axis=0)
    avg_distance=np.mean(LA.norm(pts-centroid, axis=1))
    return centroid,avg_distance


In [46]:
def normalizing_matrix(pts):
    c,d=normalizing_params(pts)
    n=pts.shape[1]
    if n==2:
        d=d/math.sqrt(2)
        mat=np.array([[1/d,0,-c[0]/d],[0,1/d,-c[1]/d],[0,0,1]])
    else:
        d=d/math.sqrt(3)
        mat=np.array([[1/d,0,0,-c[0]/d],[0,1/d,0,-c[1]/d],[0,0,1/d,-c[2]/d],[0,0,0,1]])
    return mat

In [47]:
def homo_coord(pts):
    rows=pts.shape[0]
    pts=np.append(pts, np.ones((rows, 1)), axis=1)
    return pts

In [48]:
def Fiducial_matrix(X_pts,x_pts):
    rows=X_pts.shape[0]
    X_pts_homo=homo_coord(X_pts)
    x_pts_homo=homo_coord(x_pts)

    T=normalizing_matrix(x_pts)
    U=normalizing_matrix(X_pts)
    # print(T)
    # print(U)
    x_pts_homo=(np.dot(T,x_pts_homo.T)).T
    X_pts_homo=(np.dot(U,X_pts_homo.T)).T

    
    zerocol=np.zeros((rows,4))
    multx=(-X_pts_homo.T*x_pts_homo[:,0]).T
    multy=(-X_pts_homo.T*x_pts_homo[:,1]).T

    P=np.append(X_pts_homo,zerocol,axis=1)
    P=np.append(P,multx,axis=1)
    P=np.append(P,zerocol,axis=1)
    P=np.append(P,X_pts_homo,axis=1)
    P=np.append(P,multy,axis=1)
    
    P=np.reshape(P,(2*rows,12))
    return P

In [49]:
# Fiducial_matrix(X,x)

In [50]:
def denormalize(X_pts,x_pts,P):
    T=normalizing_matrix(x_pts)
    U=normalizing_matrix(X_pts)
    P=np.dot(inv(T),P)
    P=np.dot(P,U)
    return P

In [51]:
def P_estimate(X_pts,x_pts):
    P=Fiducial_matrix(X_pts,x_pts)
    u, s, vh = np.linalg.svd(P)
    least_val_vector=vh[-1]
    P=np.reshape(least_val_vector,(3,4))
    P=denormalize(X_pts,x_pts,P)
    return P

In [52]:
def P_decomposition(P):
    KR=P[:,0:3]
    K, R = rq(KR)
    KRX=P[:,-1]
    X_0 = np.dot(-np.linalg.inv(KR),KRX)
    return K,R,X_0
    # print(r)
    # print(q)

In [53]:
# print(normalizing_matrix(x))#T
# print(normalizing_matrix(X))#U

In [54]:
P=P_estimate(X,x)
print(P)

[[ 2.56592115e+01  4.60951586e+01  1.07832658e+01  6.18561828e+01]
 [-1.07943168e+01  3.05750883e+01 -4.42653005e+01  4.42695680e+02]
 [ 3.14083369e-02  9.12628459e-03 -2.50249878e-02  4.95793551e-01]]


In [55]:
X_test = np.array([
    [0, 0, 0],
    [11.85, 9.48, 0],
    [4.74, 14.22, 0],
    [11.85, 14.22, 7.11],
    [9.48, 9.48, 0],
    [11.85, 9.48, 2.37]
]
)

In [56]:
x_actual = np.array([
    [124, 887],
    [542, 750],
    [632, 980],
    [789, 694],
    [530, 798],
    [580, 705]
])

In [57]:
def test_pred(P, X_test):
  rows=X_test.shape[0]
  X_test = np.append(X_test, np.ones((rows,1)), axis=1)
  # print(X_test)
  x_test = np.dot(P,X_test.T)
  x_test = x_test/x_test[2]
  x_test = x_test[0:2].T
  return x_test

In [58]:
# x_predict=test_pred(P,X_test)
x_predict=test_pred(P,X)

In [59]:
x_predict

array([[356.00306569, 835.99987349],
       [580.99523247, 886.00359416],
       [681.96049992, 698.73115297],
       [527.99593102, 531.99922702],
       [789.02577589, 694.65017347],
       [580.01989541, 702.6167967 ]])

In [60]:
def rms_error(x_predict, x_actual):
    return np.sqrt(np.mean((x_predict-x_actual)**2))

In [61]:
# rms_error(x_predict,x_actual)
rms_error(x,x_predict)

0.4486850397607956

In [62]:
def params1(P):
    K,R,X0=P_decomposition(P)
    alpha=K[0][0]
    theta=np.arctan(-alpha/K[0][1])
    theta_degree=theta*180/np.pi
    beta=K[1][1]*np.sin(theta)
    x0=K[0][2]
    y0=K[1][2]
    skew=K[0][1]
    t=-np.dot(R,X0)
    return alpha,beta,theta_degree,x0,y0,skew,R,X0,K,t
    
    

In [63]:
alpha,beta,theta_degree,x0,y0,skew,R,X0,K,t=params1(P)

In [92]:
K

array([[ 4.85592318e+01, -1.31695140e+00,  2.32315357e+01],
       [ 0.00000000e+00, -4.86159460e+01,  2.54413036e+01],
       [ 0.00000000e+00,  0.00000000e+00,  4.11827964e-02]])

In [64]:
X0

array([-6.61574648, -0.32386539, 11.39054574])

In [65]:
t

array([-4.56185248, -2.80590272, 12.03885104])

In [66]:
def test_pred2(P,X_test):
    rows=X_test.shape[0]
    X_test = np.append(X_test, np.ones((rows,1)), axis=1)
    # print(X_test)
    x_test = np.dot(P,X_test.T)
    x_test = x_test/x_test[2]
    return x_test

In [67]:
# arr=radial_dist(P,X).T


In [68]:
# arr

In [69]:
# x_one_pred=arr[:,0]
# print(x_one_pred)
# x_one_actual=x.T[:,0]
# print(x_one_actual)
# d2=LA.norm(np.dot(K_inv,x_one_pred))-1
# print(d2)


In [70]:
# ratio_x_one=x_one_pred[:-1]/x_one_actual
# print(ratio_x_one)

In [71]:
# meanval=np.mean(ratio_x_one)
# polyval=meanval-1
# print(polyval)

In [72]:
# rad_dist_params=3   #<=6
# poly_val=np.zeros(rad_dist_params)
# distance_power_matrix=np.zeros((rad_dist_params,rad_dist_params))
# K_inv=inv(K)
# for i in range(rad_dist_params):
#     x_one_pred=arr[:,i]
#     # print(x_one_pred)
#     x_one_actual=x.T[:,i]
#     # print(x_one_actual)
#     d2=LA.norm(np.dot(K_inv,x_one_pred))-1
#     # print(d2)
#     for j in range(rad_dist_params):
#         distance_power_matrix[j][i]=np.power(d2,j+1)
#     ratio_x_one=x_one_pred[:-1]/x_one_actual
#     # print(ratio_x_one)
#     meanval=np.mean(ratio_x_one)
#     # print(meanval)
#     polyval=meanval-1
#     poly_val[i]=polyval
# dist_inv=inv(distance_power_matrix)
# param_val=np.dot(poly_val,dist_inv)

In [73]:
# param_val

In [74]:
arr=test_pred2(P,X)
print(arr)

[[356.00306569 580.99523247 681.96049992 527.99593102 789.02577589
  580.01989541]
 [835.99987349 886.00359416 698.73115297 531.99922702 694.65017347
  702.6167967 ]
 [  1.           1.           1.           1.           1.
    1.        ]]


In [75]:
def get_param_val(P,rad_dist_params,K_inv):
    poly_val=np.zeros(rad_dist_params)
    distance_power_matrix=np.zeros((rad_dist_params,rad_dist_params))
    for i in range(rad_dist_params):
        x_one_pred=arr[:,i]
        x_one_actual=x.T[:,i]
        d2=LA.norm(np.dot(K_inv,x_one_pred))-1
        
        for j in range(rad_dist_params):
            distance_power_matrix[j][i]=np.power(d2,j+1)
        ratio_x_one=x_one_pred[:-1]/x_one_actual
        meanval=np.mean(ratio_x_one)
        polyval=meanval-1
        poly_val[i]=polyval
    dist_inv=inv(distance_power_matrix)
    param_val=np.dot(poly_val,dist_inv)
    return param_val

In [76]:
def rad_test_pred(P,X_test,rad_dist_params):


    # rad_dist_params=3   #<=6
    
    K,R,X0=P_decomposition(P)
    K_inv=inv(K)
    param_val=get_param_val(P,rad_dist_params,K_inv)


    rows=X_test.shape[0]
    X_test = np.append(X_test, np.ones((rows,1)), axis=1)
    x_test = np.dot(P,X_test.T)
    x_test = x_test/x_test[2]

    K_inv=inv(K)
    cols=x_test.shape[1]
    for i in range(cols):
        d2=LA.norm(np.dot(K_inv,x_test[:,i]))-1
        mult=1
        lamb=1
        for j in range(rad_dist_params):
            mult*=d2
            lamb+=mult*param_val[j]
        x_test[0][i]/=lamb
        x_test[1][i]/=lamb
    return x_test[:2,:].T

In [89]:
for i in range(1,7):
    print(i,rms_error(x,rad_test_pred(P,X,i)))

1 0.4486888407704048
2 0.44816655017785173
3 0.6719052177159313
4 0.45200204468778393
5 0.3151800517443621
6 0.30410394229176685


In [78]:
X_test = np.array([
    [2, 6, 0],
    [4, 4, 0]
]
)

In [79]:
x_actual = np.array([
    [632, 980],
    [530, 798]
])

In [80]:
rms_error(x_actual,test_pred(P,X_test))

4.1030328638098

In [87]:
rms_error(x_actual,rad_test_pred(P,X_test,3))

4.329204575374827

In [91]:
for i in range(1,7):
    print(i,rms_error(x_actual,rad_test_pred(P,X_test,i)))

1 4.101952259640936
2 4.091576193983036
3 4.329204575374827
4 8.42042120528208
5 19.76476407890678
6 150.59429144873513


In [95]:
get_param_val(P,2,inv(K))

array([-3.45678842e-05,  1.44261823e-06])